<a href="https://colab.research.google.com/github/bravoeight98/RelGAN_Implemetation/blob/main/RelGAN_master_implemetation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Preprocesing

In [2]:
import sys
import numpy as np

import argparse
parser = argparse.ArgumentParser()
parser.add_argument("-n", "--number", type=int, default=17, help="attribute number")
parser.add_argument("-o", "--output", type=str, default='anno_dic.npy', help="output file")
parser.add_argument("-f", "--file", required=False) 
args = parser.parse_args()

annos = open('list_attr_celeba.txt').readlines()

attrs = str.split(annos[1])
print(attrs)

if args.number == 17:
    new_attrs = ['5_o_Clock_Shadow', 'Bald', 'Bangs', 'Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Male', 'Mustache', 'Pale_Skin', 'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Hat', 'Young']
elif  args.number == 9:
    new_attrs = ['Bangs', 'Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Male', 'Mustache', 'Pale_Skin', 'Smiling', 'Young']
elif  args.number == 5:
    new_attrs = ['Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Male', 'Young']
else:
    print('You can only choose 17, 9, 5  combination')
    exit()

new_attrs_index = []
for x in new_attrs:
    new_attrs_index.append(attrs.index(x))
print(new_attrs_index)

annosAry = {}
for i in range(2,len(annos)):
    anno = str.split(annos[i])
    temp = [(int(i)+1)/2 for i in anno[1:]]
    temp2 = []
    for ii in new_attrs_index:
        temp2.append(temp[ii])
    annosAry[anno[0]] = temp2
    
print(annosAry["000001.jpg"])
print(len(annosAry["000001.jpg"]))

np.save(args.output, annosAry)

img_list = open('image_list.txt').readlines()
imgIndex = [None]*len(img_list)

for i in range(1,len(img_list)):
    temp = str.split(img_list[i])
    imgIndex[int(temp[0])] = temp[2]
    
print(imgIndex[29999])

np.save("imgIndex.npy", imgIndex)

['5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace', 'Wearing_Necktie', 'Young']
[0, 4, 5, 8, 9, 11, 15, 16, 17, 20, 22, 26, 31, 32, 33, 35, 39]
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0]
17
116055.jpg


Contrib.ops

In [3]:
import tensorflow as tf
from tensorflow import keras
#from keras.engine import Layer, InputSpec
from tensorflow.keras.layers import Layer, InputSpec
from keras import initializers
from keras import regularizers
from keras import constraints
from keras import backend as K

class GroupNormalization(Layer):
    """Group normalization layer
    Group Normalization divides the channels into groups and computes within each group
    the mean and variance for normalization. GN's computation is independent of batch sizes,
    and its accuracy is stable in a wide range of batch sizes
    # Arguments
        groups: Integer, the number of groups for Group Normalization.
        axis: Integer, the axis that should be normalized
            (typically the features axis).
            For instance, after a `Conv2D` layer with
            `data_format="channels_first"`,
            set `axis=1` in `BatchNormalization`.
        epsilon: Small float added to variance to avoid dividing by zero.
        center: If True, add offset of `beta` to normalized tensor.
            If False, `beta` is ignored.
        scale: If True, multiply by `gamma`.
            If False, `gamma` is not used.
            When the next layer is linear (also e.g. `nn.relu`),
            this can be disabled since the scaling
            will be done by the next layer.
        beta_initializer: Initializer for the beta weight.
        gamma_initializer: Initializer for the gamma weight.
        beta_regularizer: Optional regularizer for the beta weight.
        gamma_regularizer: Optional regularizer for the gamma weight.
        beta_constraint: Optional constraint for the beta weight.
        gamma_constraint: Optional constraint for the gamma weight.
    # Input shape
        Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
    # Output shape
        Same shape as input.
    # References
        - [Group Normalization](https://arxiv.org/abs/1803.08494)
    """

    def __init__(self,
                 groups=32,
                 axis=-1,
                 epsilon=1e-5,
                 center=True,
                 scale=True,
                 beta_initializer='zeros',
                 gamma_initializer='ones',
                 beta_regularizer=None,
                 gamma_regularizer=None,
                 beta_constraint=None,
                 gamma_constraint=None,
                 **kwargs):
        super(GroupNormalization, self).__init__(**kwargs)
        self.supports_masking = True
        self.groups = groups
        self.axis = axis
        self.epsilon = epsilon
        self.center = center
        self.scale = scale
        self.beta_initializer = initializers.get(beta_initializer)
        self.gamma_initializer = initializers.get(gamma_initializer)
        self.beta_regularizer = regularizers.get(beta_regularizer)
        self.gamma_regularizer = regularizers.get(gamma_regularizer)
        self.beta_constraint = constraints.get(beta_constraint)
        self.gamma_constraint = constraints.get(gamma_constraint)

    def build(self, input_shape):
        dim = input_shape[self.axis]

        if dim is None:
            raise ValueError('Axis ' + str(self.axis) + ' of '
                             'input tensor should have a defined dimension '
                             'but the layer received an input with shape ' +
                             str(input_shape) + '.')

        if dim < self.groups:
            raise ValueError('Number of groups (' + str(self.groups) + ') cannot be '
                             'more than the number of channels (' +
                             str(dim) + ').')

        if dim % self.groups != 0:
            raise ValueError('Number of groups (' + str(self.groups) + ') must be a '
                             'multiple of the number of channels (' +
                             str(dim) + ').')

        self.input_spec = InputSpec(ndim=len(input_shape),
                                    axes={self.axis: dim})
        shape = (dim,)

        if self.scale:
            self.gamma = self.add_weight(shape=shape,
                                         name='gamma',
                                         initializer=self.gamma_initializer,
                                         regularizer=self.gamma_regularizer,
                                         constraint=self.gamma_constraint)
        else:
            self.gamma = None
        if self.center:
            self.beta = self.add_weight(shape=shape,
                                        name='beta',
                                        initializer=self.beta_initializer,
                                        regularizer=self.beta_regularizer,
                                        constraint=self.beta_constraint)
        else:
            self.beta = None
        self.built = True

    def call(self, inputs, **kwargs):
        input_shape = K.int_shape(inputs)
        tensor_input_shape = K.shape(inputs)

        # Prepare broadcasting shape.
        reduction_axes = list(range(len(input_shape)))
        del reduction_axes[self.axis]
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis] // self.groups
        broadcast_shape.insert(1, self.groups)

        reshape_group_shape = K.shape(inputs)
        group_axes = [reshape_group_shape[i] for i in range(len(input_shape))]
        group_axes[self.axis] = input_shape[self.axis] // self.groups
        group_axes.insert(1, self.groups)

        # reshape inputs to new group shape
        group_shape = [group_axes[0], self.groups] + group_axes[2:]
        group_shape = K.stack(group_shape)
        inputs = K.reshape(inputs, group_shape)

        group_reduction_axes = list(range(len(group_axes)))
        group_reduction_axes = group_reduction_axes[2:]

        mean = K.mean(inputs, axis=group_reduction_axes, keepdims=True)
        variance = K.var(inputs, axis=group_reduction_axes, keepdims=True)

        inputs = (inputs - mean) / (K.sqrt(variance + self.epsilon))

        # prepare broadcast shape
        inputs = K.reshape(inputs, group_shape)
        outputs = inputs

        # In this case we must explicitly broadcast all parameters.
        if self.scale:
            broadcast_gamma = K.reshape(self.gamma, broadcast_shape)
            outputs = outputs * broadcast_gamma

        if self.center:
            broadcast_beta = K.reshape(self.beta, broadcast_shape)
            outputs = outputs + broadcast_beta

        outputs = K.reshape(outputs, tensor_input_shape)

        return outputs

    def get_config(self):
        config = {
            'groups': self.groups,
            'axis': self.axis,
            'epsilon': self.epsilon,
            'center': self.center,
            'scale': self.scale,
            'beta_initializer': initializers.serialize(self.beta_initializer),
            'gamma_initializer': initializers.serialize(self.gamma_initializer),
            'beta_regularizer': regularizers.serialize(self.beta_regularizer),
            'gamma_regularizer': regularizers.serialize(self.gamma_regularizer),
            'beta_constraint': constraints.serialize(self.beta_constraint),
            'gamma_constraint': constraints.serialize(self.gamma_constraint)
        }
        base_config = super(GroupNormalization, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape
    
from keras.layers import BatchNormalization

class SwitchNormalization(Layer):
    """Switchable Normalization layer
    Switch Normalization performs Instance Normalization, Layer Normalization and Batch
    Normalization using its parameters, and then weighs them using learned parameters to
    allow different levels of interaction of the 3 normalization schemes for each layer.
    Only supports the moving average variant from the paper, since the `batch average`
    scheme requires dynamic graph execution to compute the mean and variance of several
    batches at runtime.
    # Arguments
        axis: Integer, the axis that should be normalized
            (typically the features axis).
            For instance, after a `Conv2D` layer with
            `data_format="channels_first"`,
            set `axis=1` in `BatchNormalization`.
        momentum: Momentum for the moving mean and the moving variance. The original
            implementation suggests a default momentum of `0.997`, however it is highly
            unstable and training can fail after a few epochs. To stabilise training, use
            lower values of momentum such as `0.99` or `0.98`.
        epsilon: Small float added to variance to avoid dividing by zero.
        final_gamma: Bool value to determine if this layer is the final
            normalization layer for the residual block.  Overrides the initialization
            of the scaling weights to be `zeros`. Only used for Residual Networks,
            to make the forward/backward signal initially propagated through an
            identity shortcut.
        center: If True, add offset of `beta` to normalized tensor.
            If False, `beta` is ignored.
        scale: If True, multiply by `gamma`.
            If False, `gamma` is not used.
            When the next layer is linear (also e.g. `nn.relu`),
            this can be disabled since the scaling
            will be done by the next layer.
        beta_initializer: Initializer for the beta weight.
        gamma_initializer: Initializer for the gamma weight.
        mean_weights_initializer: Initializer for the mean weights.
        variance_weights_initializer: Initializer for the variance weights.
        moving_mean_initializer: Initializer for the moving mean.
        moving_variance_initializer: Initializer for the moving variance.
        beta_regularizer: Optional regularizer for the beta weight.
        gamma_regularizer: Optional regularizer for the gamma weight.
        mean_weights_regularizer: Optional regularizer for the mean weights.
        variance_weights_regularizer: Optional regularizer for the variance weights.
        beta_constraint: Optional constraint for the beta weight.
        gamma_constraint: Optional constraint for the gamma weight.
        mean_weights_constraints: Optional constraint for the mean weights.
        variance_weights_constraints: Optional constraint for the variance weights.
    # Input shape
        Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
    # Output shape
        Same shape as input.
    # References
        - [Differentiable Learning-to-Normalize via Switchable Normalization](https://arxiv.org/abs/1806.10779)
    """

    def __init__(self,
                 axis=-1,
                 momentum=0.9,
                 epsilon=1e-5,
                 final_gamma=False,
                 center=True,
                 scale=True,
                 beta_initializer='zeros',
                 gamma_initializer='ones',
                 mean_weights_initializer='ones',
                 variance_weights_initializer='ones',
                 moving_mean_initializer='ones',
                 moving_variance_initializer='zeros',
                 beta_regularizer=None,
                 gamma_regularizer=None,
                 mean_weights_regularizer=None,
                 variance_weights_regularizer=None,
                 beta_constraint=None,
                 gamma_constraint=None,
                 mean_weights_constraints=None,
                 variance_weights_constraints=None,
                 **kwargs):
        super(SwitchNormalization, self).__init__(**kwargs)
        self.supports_masking = True
        self.axis = axis
        self.momentum = momentum
        self.epsilon = epsilon
        self.center = center
        self.scale = scale

        self.beta_initializer = initializers.get(beta_initializer)
        if final_gamma:
            self.gamma_initializer = initializers.get('zeros')
        else:
            self.gamma_initializer = initializers.get(gamma_initializer)
        self.mean_weights_initializer = initializers.get(mean_weights_initializer)
        self.variance_weights_initializer = initializers.get(variance_weights_initializer)
        self.moving_mean_initializer = initializers.get(moving_mean_initializer)
        self.moving_variance_initializer = initializers.get(moving_variance_initializer)
        self.beta_regularizer = regularizers.get(beta_regularizer)
        self.gamma_regularizer = regularizers.get(gamma_regularizer)
        self.mean_weights_regularizer = regularizers.get(mean_weights_regularizer)
        self.variance_weights_regularizer = regularizers.get(variance_weights_regularizer)
        self.beta_constraint = constraints.get(beta_constraint)
        self.gamma_constraint = constraints.get(gamma_constraint)
        self.mean_weights_constraints = constraints.get(mean_weights_constraints)
        self.variance_weights_constraints = constraints.get(variance_weights_constraints)

    def build(self, input_shape):
        dim = input_shape[self.axis]

        if dim is None:
            raise ValueError('Axis ' + str(self.axis) + ' of '
                             'input tensor should have a defined dimension '
                             'but the layer received an input with shape ' +
                             str(input_shape) + '.')

        self.input_spec = InputSpec(ndim=len(input_shape),
                                    axes={self.axis: dim})
        shape = (dim,)

        if self.scale:
            self.gamma = self.add_weight(
                shape=shape,
                name='gamma',
                initializer=self.gamma_initializer,
                regularizer=self.gamma_regularizer,
                constraint=self.gamma_constraint)
        else:
            self.gamma = None
        if self.center:
            self.beta = self.add_weight(
                shape=shape,
                name='beta',
                initializer=self.beta_initializer,
                regularizer=self.beta_regularizer,
                constraint=self.beta_constraint)
        else:
            self.beta = None

        self.moving_mean = self.add_weight(
            shape=shape,
            name='moving_mean',
            initializer=self.moving_mean_initializer,
            trainable=False)

        self.moving_variance = self.add_weight(
            shape=shape,
            name='moving_variance',
            initializer=self.moving_variance_initializer,
            trainable=False)

        self.mean_weights = self.add_weight(
            shape=(3,),
            name='mean_weights',
            initializer=self.mean_weights_initializer,
            regularizer=self.mean_weights_regularizer,
            constraint=self.mean_weights_constraints)

        self.variance_weights = self.add_weight(
            shape=(3,),
            name='variance_weights',
            initializer=self.variance_weights_initializer,
            regularizer=self.variance_weights_regularizer,
            constraint=self.variance_weights_constraints)

        self.built = True

    def call(self, inputs, training=None):
        input_shape = K.int_shape(inputs)

        # Prepare broadcasting shape.
        reduction_axes = list(range(len(input_shape)))
        del reduction_axes[self.axis]

        if self.axis != 0:
            del reduction_axes[0]

        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis]

        mean_instance = K.mean(inputs, reduction_axes, keepdims=True)
        variance_instance = K.var(inputs, reduction_axes, keepdims=True)

        mean_layer = K.mean(mean_instance, self.axis, keepdims=True)
        temp = variance_instance + K.square(mean_instance)
        variance_layer = K.mean(temp, self.axis, keepdims=True) - K.square(mean_layer)

        def training_phase():
            mean_batch = K.mean(mean_instance, axis=0, keepdims=True)
            variance_batch = K.mean(temp, axis=0, keepdims=True) - K.square(mean_batch)

            mean_batch_reshaped = K.flatten(mean_batch)
            variance_batch_reshaped = K.flatten(variance_batch)

            if K.backend() != 'cntk':
                sample_size = K.prod([K.shape(inputs)[axis]
                                      for axis in reduction_axes])
                sample_size = K.cast(sample_size, dtype=K.dtype(inputs))

                # sample variance - unbiased estimator of population variance
                variance_batch_reshaped *= sample_size / (sample_size - (1.0 + self.epsilon))

            self.add_update([K.moving_average_update(self.moving_mean,
                                                     mean_batch_reshaped,
                                                     self.momentum),
                             K.moving_average_update(self.moving_variance,
                                                     variance_batch_reshaped,
                                                     self.momentum)],
                            inputs)

            return normalize_func(mean_batch, variance_batch)

        def inference_phase():
            mean_batch = self.moving_mean
            variance_batch = self.moving_variance

            return normalize_func(mean_batch, variance_batch)

        def normalize_func(mean_batch, variance_batch):
            mean_batch = K.reshape(mean_batch, broadcast_shape)
            variance_batch = K.reshape(variance_batch, broadcast_shape)

            mean_weights = K.softmax(self.mean_weights, axis=0)
            variance_weights = K.softmax(self.variance_weights, axis=0)

            mean = (mean_weights[0] * mean_instance +
                    mean_weights[1] * mean_layer +
                    mean_weights[2] * mean_batch)

            variance = (variance_weights[0] * variance_instance +
                        variance_weights[1] * variance_layer +
                        variance_weights[2] * variance_batch)

            outputs = (inputs - mean) / (K.sqrt(variance + self.epsilon))

            if self.scale:
                broadcast_gamma = K.reshape(self.gamma, broadcast_shape)
                outputs = outputs * broadcast_gamma

            if self.center:
                broadcast_beta = K.reshape(self.beta, broadcast_shape)
                outputs = outputs + broadcast_beta

            return outputs

        if training in {0, False}:
            return inference_phase()

        return K.in_train_phase(training_phase,
                                inference_phase,
                                training=training)

    def get_config(self):
        config = {
            'axis': self.axis,
            'epsilon': self.epsilon,
            'momentum': self.momentum,
            'center': self.center,
            'scale': self.scale,
            'beta_initializer': initializers.serialize(self.beta_initializer),
            'gamma_initializer': initializers.serialize(self.gamma_initializer),
            'mean_weights_initializer': initializers.serialize(self.mean_weights_initializer),
            'variance_weights_initializer': initializers.serialize(self.variance_weights_initializer),
            'moving_mean_initializer': initializers.serialize(self.moving_mean_initializer),
            'moving_variance_initializer': initializers.serialize(self.moving_variance_initializer),
            'beta_regularizer': regularizers.serialize(self.beta_regularizer),
            'gamma_regularizer': regularizers.serialize(self.gamma_regularizer),
            'mean_weights_regularizer': regularizers.serialize(self.mean_weights_regularizer),
            'variance_weights_regularizer': regularizers.serialize(self.variance_weights_regularizer),
            'beta_constraint': constraints.serialize(self.beta_constraint),
            'gamma_constraint': constraints.serialize(self.gamma_constraint),
            'mean_weights_constraints': constraints.serialize(self.mean_weights_constraints),
            'variance_weights_constraints': constraints.serialize(self.variance_weights_constraints),
        }
        base_config = super(SwitchNormalization, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape
    
class InstanceNormalization(Layer):
    """Instance normalization layer.
    Normalize the activations of the previous layer at each step,
    i.e. applies a transformation that maintains the mean activation
    close to 0 and the activation standard deviation close to 1.
    # Arguments
        axis: Integer, the axis that should be normalized
            (typically the features axis).
            For instance, after a `Conv2D` layer with
            `data_format="channels_first"`,
            set `axis=1` in `InstanceNormalization`.
            Setting `axis=None` will normalize all values in each
            instance of the batch.
            Axis 0 is the batch dimension. `axis` cannot be set to 0 to avoid errors.
        epsilon: Small float added to variance to avoid dividing by zero.
        center: If True, add offset of `beta` to normalized tensor.
            If False, `beta` is ignored.
        scale: If True, multiply by `gamma`.
            If False, `gamma` is not used.
            When the next layer is linear (also e.g. `nn.relu`),
            this can be disabled since the scaling
            will be done by the next layer.
        beta_initializer: Initializer for the beta weight.
        gamma_initializer: Initializer for the gamma weight.
        beta_regularizer: Optional regularizer for the beta weight.
        gamma_regularizer: Optional regularizer for the gamma weight.
        beta_constraint: Optional constraint for the beta weight.
        gamma_constraint: Optional constraint for the gamma weight.
    # Input shape
        Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a Sequential model.
    # Output shape
        Same shape as input.
    # References
        - [Layer Normalization](https://arxiv.org/abs/1607.06450)
        - [Instance Normalization: The Missing Ingredient for Fast Stylization](
        https://arxiv.org/abs/1607.08022)
    """
    def __init__(self,
                 axis=None,
                 epsilon=1e-3,
                 center=True,
                 scale=True,
                 beta_initializer='zeros',
                 gamma_initializer='ones',
                 beta_regularizer=None,
                 gamma_regularizer=None,
                 beta_constraint=None,
                 gamma_constraint=None,
                 **kwargs):
        super(InstanceNormalization, self).__init__(**kwargs)
        self.supports_masking = True
        self.axis = axis
        self.epsilon = epsilon
        self.center = center
        self.scale = scale
        self.beta_initializer = initializers.get(beta_initializer)
        self.gamma_initializer = initializers.get(gamma_initializer)
        self.beta_regularizer = regularizers.get(beta_regularizer)
        self.gamma_regularizer = regularizers.get(gamma_regularizer)
        self.beta_constraint = constraints.get(beta_constraint)
        self.gamma_constraint = constraints.get(gamma_constraint)

    def build(self, input_shape):
        ndim = len(input_shape)
        if self.axis == 0:
            raise ValueError('Axis cannot be zero')

        if (self.axis is not None) and (ndim == 2):
            raise ValueError('Cannot specify axis for rank 1 tensor')

        self.input_spec = InputSpec(ndim=ndim)

        if self.axis is None:
            shape = (1,)
        else:
            shape = (input_shape[self.axis],)

        if self.scale:
            self.gamma = self.add_weight(shape=shape,
                                         name='gamma',
                                         initializer=self.gamma_initializer,
                                         regularizer=self.gamma_regularizer,
                                         constraint=self.gamma_constraint)
        else:
            self.gamma = None
        if self.center:
            self.beta = self.add_weight(shape=shape,
                                        name='beta',
                                        initializer=self.beta_initializer,
                                        regularizer=self.beta_regularizer,
                                        constraint=self.beta_constraint)
        else:
            self.beta = None
        self.built = True

    def call(self, inputs, training=None):
        input_shape = K.int_shape(inputs)
        reduction_axes = list(range(0, len(input_shape)))

        if self.axis is not None:
            del reduction_axes[self.axis]

        del reduction_axes[0]

        mean = K.mean(inputs, reduction_axes, keepdims=True)
        stddev = K.std(inputs, reduction_axes, keepdims=True) + self.epsilon
        normed = (inputs - mean) / stddev

        broadcast_shape = [1] * len(input_shape)
        if self.axis is not None:
            broadcast_shape[self.axis] = input_shape[self.axis]

        if self.scale:
            broadcast_gamma = K.reshape(self.gamma, broadcast_shape)
            normed = normed * broadcast_gamma
        if self.center:
            broadcast_beta = K.reshape(self.beta, broadcast_shape)
            normed = normed + broadcast_beta
        return normed

    def get_config(self):
        config = {
            'axis': self.axis,
            'epsilon': self.epsilon,
            'center': self.center,
            'scale': self.scale,
            'beta_initializer': initializers.serialize(self.beta_initializer),
            'gamma_initializer': initializers.serialize(self.gamma_initializer),
            'beta_regularizer': regularizers.serialize(self.beta_regularizer),
            'gamma_regularizer': regularizers.serialize(self.gamma_regularizer),
            'beta_constraint': constraints.serialize(self.beta_constraint),
            'gamma_constraint': constraints.serialize(self.gamma_constraint)
        }
        base_config = super(InstanceNormalization, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

Ops

In [5]:
import tensorflow as tf
import keras
from keras.layers import LeakyReLU, Conv2D, Add, ZeroPadding2D, Activation, Lambda, Dropout
from keras import backend as K

def hard_tanh(x):
    return K.clip(x, -1, )

def orthogonal(w):
    
    w_kw = K.int_shape(w)[0]
    w_kh = K.int_shape(w)[1]
    w_w = K.int_shape(w)[2]
    w_h = K.int_shape(w)[3]
    
    temp = 0
    for i in range(w_kw):
        for j in range(w_kh):
            wwt = tf.matmul(tf.transpose(w[i,j]), w[i,j])
            mi = K.ones_like(wwt) - K.identity(wwt)
            a = wwt * mi
            a = tf.matmul(tf.transpose(a), a)
            a = a * K.identity(a)
            temp += K.sum(a)
    return 2e-6 * temp

def residual_block(x, dim, ks, init_weight, name):
    y = Conv2D(dim, ks, strides=1, padding="same", kernel_initializer=init_weight, kernel_regularizer = orthogonal)(x)
    y = SwitchNormalization(axis=-1, name=name+'_0')(y)
    y = Activation('relu')(y)   
    y = Conv2D(dim, ks, strides=1, padding="same", kernel_initializer=init_weight, kernel_regularizer = orthogonal)(y)
    y = SwitchNormalization(axis=-1, name=name+'_1')(y)
    return Add()([x,y])
    
def glu(x):
    channel = K.int_shape(x)[-1]
    channel = channel//2
    a = x[..., :channel]
    b = x[..., channel:]
    return a * K.sigmoid(b)

Module

In [6]:
import functools
import tensorflow as tf
import keras
from keras import regularizers
from keras.models import Model, Sequential
from keras.layers import LeakyReLU, Activation, Input, Reshape, Flatten, Dense, Multiply
from keras.layers import Conv2D, Conv2DTranspose, ZeroPadding2D, Lambda, Concatenate, Add
from keras.layers import BatchNormalization, Dropout, Subtract, GlobalAveragePooling2D

init_weight = 'he_normal'
res_init_weight = 'he_normal'

regular = None

def tileAttr(x):
        x = tf.expand_dims(x, axis = 1)
        x = tf.expand_dims(x, axis = 2)
        return tf.tile(x, [1, 256, 256, 1])
    
def tileAttr2(x):
        x = tf.expand_dims(x, axis = 1)
        x = tf.expand_dims(x, axis = 2)
        return tf.tile(x, [1, 4, 4, 1])

def generator(img, attr, size):
    
    concat = Concatenate()([img, Lambda(tileAttr)(attr)])
    
    DownSample = functools.partial(Conv2D, padding="same" , kernel_initializer=init_weight, kernel_regularizer = orthogonal)
    UpSample = functools.partial(Conv2DTranspose, padding="same" , kernel_initializer=init_weight, kernel_regularizer = orthogonal)
    
    conv_in = DownSample(64, 7, name="conv_in_conv")(concat)
    conv_in = SwitchNormalization(axis=-1, name="conv_in_norm")(conv_in)
    conv_in = Activation('relu', name="conv_in_relu")(conv_in)
    
    down1 = DownSample(128, 4, strides=2, name="down1_conv")(conv_in)
    down1 = SwitchNormalization(axis=-1, name="down1_norm")(down1)
    down1 = Activation('relu', name="down1_relu")(down1)
    
    down2 = DownSample(256, 4, strides=2, name="down2_conv")(down1)
    down2 = SwitchNormalization(axis=-1, name="down2_norm")(down2)
    down2 = Activation('relu', name="down2_relu")(down2)
    
    resb = residual_block(down2, 256, 3, res_init_weight, 'block1')
    resb = residual_block(resb, 256, 3, res_init_weight, 'block2')
    resb = residual_block(resb, 256, 3, res_init_weight, 'block3')
    
    encode_out = resb
    
    resb = residual_block(resb, 256, 3, res_init_weight, 'block4')
    resb = residual_block(resb, 256, 3, res_init_weight, 'block5')
    resb = residual_block(resb, 256, 3, res_init_weight, 'block6')
    
    up2 = UpSample(128, 4, strides=2, name="up2_deconv2")(resb)
    up2 = SwitchNormalization(axis=-1, name="up2_norm")(up2)
    up2 = Activation('relu', name="up2_relu")(up2)
    brid2 = up2
    
    up1 = UpSample(64 , 4, strides=2, name="up1_deconv2")(brid2)
    up1 = SwitchNormalization(axis=-1, name="up1_norm")(up1)
    up1 = Activation('relu', name="up1_relu")(up1)
    brid3 = up1
    
    conv_out = DownSample(3, 7, name="conv_out_conv")(brid3)
    conv_out = Activation('tanh', name="conv_out_tanh")(conv_out)
    return conv_out, encode_out

def discriminator(imgA, imgB, attr, size, att_size):
    
    filters = [64, 128, 256, 512, 1024, 2048]
    
    convs = [Conv2D(64, 4, strides=2, padding='same', kernel_initializer=init_weight, kernel_regularizer=regular, name="conv1"),
             Conv2D(128, 4, strides=2, padding='same', kernel_initializer=init_weight, kernel_regularizer=regular, name="conv2"),
             Conv2D(256, 4, strides=2, padding='same', kernel_initializer=init_weight, kernel_regularizer=regular, name="conv3"),
             Conv2D(512, 4, strides=2, padding='same', kernel_initializer=init_weight, kernel_regularizer=regular, name="conv4"),
             Conv2D(1024, 4, strides=2, padding='same', kernel_initializer=init_weight, kernel_regularizer=regular, name="conv5"),
             Conv2D(2048, 4, strides=2, padding='same', kernel_initializer=init_weight, kernel_regularizer=regular, name="conv6")]
    
    
    #original image
    
    y1 = imgA
    for i in range(6):
        y1 = convs[i](y1)
        y1 = LeakyReLU(alpha=0.01)(y1)
    
    #target image
    y2 = imgB
    for i in range(6):
        y2 = convs[i](y2)
        y2 = LeakyReLU(alpha=0.01)(y2)
    
    d_out1 = Conv2D(1, 1, padding='same', kernel_initializer='lecun_normal', kernel_regularizer=regular)(y2)
    
    d_out3 = Conv2D(64, 1, padding='same', kernel_initializer='lecun_normal', kernel_regularizer=regular)(y2)
    d_out3 = Lambda(lambda x: K.mean(x, axis=[-1]))(d_out3)
    
    d_out2 = Concatenate()([y1, y2, Lambda(tileAttr2)(attr)])
    d_out2 = Conv2D(2048, 1, strides=1, kernel_initializer='lecun_normal', kernel_regularizer=regular)(d_out2)
    d_out2 = LeakyReLU(alpha=0.01)(d_out2) # 2 2 2048    
    d_out2 = Conv2D(1, 1, padding='same', kernel_initializer='lecun_normal', kernel_regularizer=regular)(d_out2)
    
    return d_out1, d_out2, d_out3

Import tensorBoardx

In [7]:
!pip install tensorboardX

     |████████████████████████████████| 133kB 5.3MB/s 


RelGAN Model

In [8]:
import os
import random
import re
import time
import numpy as np
from PIL import Image
import tensorflow as tf
from keras.layers import Input
from keras.models import Model, Sequential, load_model
from keras.optimizers import Adam, RMSprop
from keras import backend as K
from tensorflow.keras.utils import plot_model
from tensorflow.python.keras.backend import set_session
from skimage import io, transform
from tensorboardX import SummaryWriter
from keras.preprocessing.image import ImageDataGenerator


class Relgan():

    def __init__(self, args):

        self.path = args.path
        self.lr = args.lr
        self.b1 = args.beta1
        self.b2 = args.beta2
        self.batch = args.batch_size
        self.sample = args.sample_size
        self.epochs = args.epochs
        self.lambda1 = args.lambda1
        self.lambda2 = args.lambda2
        self.lambda4 = args.lambda4
        self.lambda5 = args.lambda5
        self.gp_l = args.lambda_gp
        self.decay = self.lr / self.epochs
        self.imgSize = args.img_size
        self.sampleSize = args.img_size
        self.vecSize = args.vec_size
        self.step = args.step * 200

        self.lr -= self.decay * self.step

        self.img_shape = (self.imgSize, self.imgSize, 3)
        self.vec_shape = (self.vecSize,)

        self.get_model()
        self.get_loss()
        self.get_optimizer()
        self.datagen = ImageDataGenerator(horizontal_flip=True)
        self.writer = SummaryWriter()

    def get_model(self):

        self.imgA_input = Input(shape=self.img_shape)
        self.imgB_input = Input(shape=self.img_shape)
        self.vec_input_pos = Input(shape=self.vec_shape)
        self.vec_input_neg = Input(shape=self.vec_shape)

        g_out = generator(self.imgA_input, self.vec_input_pos, self.imgSize)

        self.g_model = Model(inputs=[self.imgA_input, self.vec_input_pos], outputs=g_out)

        d_out = discriminator(self.imgA_input, self.imgB_input, self.vec_input_pos, self.imgSize, self.vecSize)

        self.d_model = Model(inputs=[self.imgA_input, self.imgB_input, self.vec_input_pos], \
                             outputs=d_out)

        print(self.g_model.summary())
        print(self.d_model.summary())

        plot_model(self.g_model, to_file='g_model.png')
        plot_model(self.d_model, to_file='d_model.png')

    def get_loss(self):

        def cal_df_gp():

            def cal_gp(gradients):
                gradients_sqr = K.square(gradients[0])
                gradients_sqr_sum = K.sum(gradients_sqr, axis=np.arange(1, len(gradients_sqr.shape)))
                gradient_l2_norm = K.sqrt(gradients_sqr_sum)
                gradient_penalty = K.mean(K.square(1 - gradient_l2_norm))
                return gradient_penalty

            alpha = K.random_uniform_variable(shape=(1,), low=0, high=1)

            mix_tar = alpha * self.img_a + (1 - alpha) * self.img_a2b

            mix_outputs_a2b = self.d_model([self.img_a, mix_tar, self.vec_ab_pos])
            mix_outputs_a2ab = self.d_model([self.img_a, self.img_a2ab, self.vec_ab_pos])

            gradients_a2b = K.gradients([mix_outputs_a2b[0]], [mix_tar])
            gradients_a2ab = K.gradients([mix_outputs_a2ab[2]], [self.img_a2ab])

            df_gp = cal_gp(gradients_a2b) + cal_gp(gradients_a2ab)

            return df_gp

        def lsgan(xs, ts):
            real = 0
            fake = 0
            for i in range(len(xs)):
                if ts[i] == 1:
                    real += K.mean(K.square(K.ones_like(xs[i]) - xs[i]), axis=[-1])
                else:
                    fake += K.mean(K.square(K.zeros_like(xs[i]) - xs[i]), axis=[-1])

            return real + fake

        self.img_a = Input(shape=self.img_shape)
        self.img_b = Input(shape=self.img_shape)
        self.img_c = Input(shape=self.img_shape)

        self.vec_ab_pos = Input(shape=self.vec_shape)
        self.vec_ac_pos = Input(shape=self.vec_shape)
        self.vec_cb_pos = Input(shape=self.vec_shape)

        self.img_a2b, self.enc_a2b = self.g_model([self.img_a, self.vec_ab_pos])
        self.img_a2a, self.enc_a2a = self.g_model([self.img_a, K.zeros_like(self.vec_ab_pos)])
        self.img_a2b2a, _ = self.g_model([self.img_a2b, -self.vec_ab_pos])

        inter_seed = K.random_uniform_variable(shape=([self.batch, ]), low=0, high=1)
        inter_seed = K.reshape(inter_seed, [self.batch, 1])
        self.img_a2ab, self.enc_a2ab = self.g_model([self.img_a, inter_seed * self.vec_ab_pos])

        input_real = [self.img_a, self.img_b, self.vec_ab_pos]
        input_fake = [self.img_a, self.img_a2b, self.vec_ab_pos]
        input_w_ori = [self.img_c, self.img_b, self.vec_ab_pos]
        input_w_tar = [self.img_a, self.img_c, self.vec_ab_pos]
        input_w_vec1 = [self.img_a, self.img_b, self.vec_ac_pos]
        input_w_vec2 = [self.img_a, self.img_b, self.vec_cb_pos]

        input_inter = [self.img_a, self.img_a2ab, inter_seed * self.vec_ab_pos]
        input_zero = [self.img_a, self.img_a2a, K.zeros_like(self.vec_ab_pos)]

        d_real, dc_real, _ = self.d_model(input_real)

        d_fake, dc_fake, di_fake = self.d_model(input_fake)

        d_w_ori, dc_w_ori, _ = self.d_model(input_w_ori)
        d_w_tar, dc_w_tar, _ = self.d_model(input_w_tar)
        d_w_vec1, dc_w_vec1, _ = self.d_model(input_w_vec1)
        d_w_vec2, dc_w_vec2, _ = self.d_model(input_w_vec2)

        _, _, di_inter = self.d_model(input_inter)
        _, _, di_zero = self.d_model(input_zero)

        self.df_loss = lsgan([d_real, d_fake], [1, 0])
        self.dc_loss = lsgan([dc_real, dc_fake, dc_w_ori, dc_w_tar, dc_w_vec1, dc_w_vec2], [1, 0, 0, 0, 0, 0])
        inter_seed_rep = K.flatten(inter_seed)

        di_temp = K.switch(K.less(inter_seed_rep, 0.5 * K.ones_like(inter_seed_rep)), di_zero, di_fake)

        self.di_loss = K.square(K.minimum(inter_seed_rep, K.ones_like(inter_seed_rep) - inter_seed_rep) * K.ones_like(
            di_inter) - di_inter) + K.square(di_temp)
        print('self.df_loss', K.int_shape(self.df_loss))
        print('self.dc_loss', K.int_shape(self.dc_loss))
        print('self.di_loss', K.int_shape(self.di_loss))

        self.df_gp = cal_df_gp()

        self.d_loss = self.df_loss + self.dc_loss + self.gp_l * self.df_gp + self.lambda5 * self.di_loss

        self.gf_loss = lsgan([d_real, d_fake], [0, 1])
        self.gc_loss = lsgan([dc_real, dc_fake], [0, 1])
        self.gi_loss = K.square(di_inter)

        dist_a2b = self.enc_a2b - self.enc_a2a
        dist_a2ab = self.enc_a2ab - self.enc_a2a

        inter_seed = K.reshape(inter_seed, [self.batch, 1, 1, 1])
        self.g_inter_loss = K.mean(K.abs(inter_seed * dist_a2b - dist_a2ab))

        g_loss_rec1 = K.mean(K.abs(self.img_a - self.img_a2b2a))
        g_loss_rec2 = K.mean(K.abs(self.img_a - self.img_a2a))

        print('self.gf_loss', K.int_shape(self.gf_loss))
        print('self.gc_loss', K.int_shape(self.gc_loss))
        print('self.gi_loss', K.int_shape(self.gi_loss))
        print('self.g_loss_rec1', K.int_shape(g_loss_rec1))
        print('self.g_loss_rec2', K.int_shape(g_loss_rec2))

        self.gr_loss = self.lambda1 * g_loss_rec1 + self.lambda2 * g_loss_rec2
        self.g_loss = self.gf_loss + self.gc_loss + self.gr_loss + self.lambda5 * self.gi_loss

    def get_optimizer(self):

        g_opt = Adam(lr=self.lr, decay=self.decay, beta_1=self.b1, beta_2=self.b2)
        g_weights = self.g_model.trainable_weights
        g_inputs = [self.img_a, self.img_b, self.vec_ab_pos]

        self.g_training_updates = g_opt.get_updates(g_weights, [], self.g_loss)
        self.g_train = K.function(g_inputs,
                                  [K.mean(self.g_loss),
                                   K.mean(self.gf_loss),
                                   K.mean(self.gc_loss),
                                   K.mean(self.gr_loss),
                                   K.mean(self.g_inter_loss),
                                   K.mean(self.gi_loss)],
                                  self.g_training_updates)

        d_opt = Adam(lr=self.lr, decay=self.decay, beta_1=self.b1, beta_2=self.b2)
        d_weights = self.d_model.trainable_weights
        d_inputs = [self.img_a, self.img_b, self.img_c, self.vec_ab_pos, self.vec_ac_pos, self.vec_cb_pos]

        self.d_training_updates = d_opt.get_updates(d_weights, [], self.d_loss)
        self.d_train = K.function(d_inputs,
                                  [K.mean(self.d_loss),
                                   K.mean(self.df_loss),
                                   K.mean(self.dc_loss),
                                   K.mean(self.gp_l * self.df_gp),
                                   K.mean(self.di_loss)],
                                  self.d_training_updates)

    def get_imgs_tags(self, indexserX, imgIndex, imgAttr):
        imgs = [None] * self.batch
        atts = [None] * self.batch

        for i in range(self.batch):
            temp_index = indexserX[i]
        #    temp_index = int(re.sub("\D", "", temp_index ))
            print(temp_index)
            x = np.where(imgIndex == temp_index)
            print(x)
            print(indexserX[i])                        #
            img_fa = imgIndex[int(x[0])]

      #      while img_fa == None:
       #         temp_index = np.random.choice((imgIndex), 1)[0]    #
        #        img_fa = imgIndex[temp_index]
            atts[i] = imgAttr[img_fa]
            print(temp_index)                                           #
            img = io.imread(os.path.join(self.path, str(temp_index).zfill(10)))
            imgs[i] = img / 127.5 - 1

        imgs = np.array(imgs)
        atts = np.array(atts)

        self.datagen.fit(imgs)

        imgs = self.datagen.flow(imgs, batch_size=self.batch, shuffle=False).next()

        return imgs, atts

    def train(self):

        print("load index")
        imgIndex = np.load("imgIndex.npy")
        imgAttr = np.load("anno_dic.npy").item()
        print("training")

        ite = self.step

        def getIndex():
            while True:
                count = 0
                index_permutation = np.random.permutation((imgIndex))     #
                while count + self.batch * 3 < len(imgIndex):
                    yield index_permutation[count:(count + self.batch * 3)]
                    count = count + self.batch * 3

        index_gen = getIndex()

        def get_training_data(wrong=False):
            indexser = next(index_gen)
            indexser1 = indexser[self.batch * 0:self.batch * 1]
            indexser2 = indexser[self.batch * 1:self.batch * 2]
            indexser3 = indexser[self.batch * 2:self.batch * 3]

            img_as, att_as = self.get_imgs_tags(indexser1, imgIndex, imgAttr)
            img_bs, att_bs = self.get_imgs_tags(indexser2, imgIndex, imgAttr)
            vec_ab_pos = att_bs - att_as

            if wrong == False:
                return img_as, img_bs, vec_ab_pos

            img_cs, att_cs = self.get_imgs_tags(indexser3, imgIndex, imgAttr)

            vec_ac_pos = att_cs - att_as
            vec_cb_pos = att_bs - att_cs

            return img_as, img_bs, img_cs, vec_ab_pos, vec_ac_pos, vec_cb_pos

        for ep in range(int(self.epochs)):

            t_start = time.time()

            img_as, img_bs, img_cs, vec_ab_pos, vec_ac_pos, vec_cb_pos = get_training_data(wrong=True)

            for i in range(1):
                errD = self.d_train([img_as, img_bs, img_cs, vec_ab_pos, vec_ac_pos, vec_cb_pos])

            for i in range(1):
                errG = self.g_train([img_as, img_bs, vec_ab_pos])

            t_end = time.time()

            print(
                "%9.6f %9.6f | real: %7.4f wrong: %7.4f gp: %7.4f| fake: %7.4f wrong: %7.4f recs: %7.4f enc: %7.4f| time: %.4f" % (
                errD[0], errG[0], errD[1], errD[2], errD[3], errG[1], errG[2], errG[3], errG[4], t_end - t_start))

            self.writer.add_scalar('d_loss', errD[0], ite)
            self.writer.add_scalar('g_loss', errG[0], ite)
            self.writer.add_scalar('df_loss', errD[1], ite)
            self.writer.add_scalar('gf_loss', errG[1], ite)
            self.writer.add_scalar('dc_loss', errD[2], ite)
            self.writer.add_scalar('gc_loss', errG[2], ite)
            self.writer.add_scalar('gr_loss', errG[3], ite)
            self.writer.add_scalar('inter_loss', errG[4], ite)
            self.writer.add_scalar('gp_loss', errD[3], ite)
            self.writer.add_scalar('gi_loss', errG[5], ite)
            self.writer.add_scalar('di_loss', errD[4], ite)

            if ite % 50 == 0 and ite > 0:

                img_as, img_bs, vec_ab_pos = get_training_data(wrong=False)

                g_a2b = [img_as[:self.sample], vec_ab_pos[:self.sample]]
                fakea2b, _ = self.g_model.predict(g_a2b)

                g_a2a = [img_as[:self.sample], np.zeros([self.sample, self.vecSize])]
                fakea2a, _ = self.g_model.predict(g_a2a)

                g_a2b2a = [fakea2b[:self.sample], -vec_ab_pos[:self.sample]]
                fakea2b2a, _ = self.g_model.predict(g_a2b2a)

                images = np.concatenate([img_as[:self.sample], fakea2b, fakea2b2a, fakea2a], axis=0)

                width = self.sample
                height = 4
                new_im = Image.new('RGB', (self.sampleSize * height, self.sampleSize * width))
                for ii in range(height):
                    for jj in range(width):
                        index = ii * width + jj
                        image = (images[index] / 2 + 0.5) * 255
                        image = transform.resize(image, (self.sampleSize, self.sampleSize), preserve_range=True)
                        #                         image = image*255
                        image = image.astype(np.uint8)
                        new_im.paste(Image.fromarray(image, "RGB"), (self.sampleSize * ii, self.sampleSize * jj))
                filename = "img/fakeFace%d.jpg" % (ite // 200)
                new_im.save(filename)

                try:
                    self.g_model.save("model/generator%d.h5" % (ite // 200))
                    self.d_model.save("model/discriminator.h5")
                except:
                    print('Pass save')
            ite = ite + 1

Train

In [16]:
import argparse
parser = argparse.ArgumentParser()
#parser.add_argument("-p", "--path", type=str, default='E:\Current_Sem_Project\CSE465\Dataset\celeba-hq\celeba-256', help="data path")
parser.add_argument("-p", "--path", type=str, default='/content/drive/RelGAN_master_implemetation/celebA_images/', help="data path")
parser.add_argument("-d", "--device", type=str, default='0', help="gpu device")
parser.add_argument("-g", "--growth", type=bool, default=False, help="allow_growth")
parser.add_argument("-s", "--step", type=int, default=0, help="train_step")
parser.add_argument("-l", "--lr", type=float, default=5e-5)
parser.add_argument("-b1", "--beta1", type=float, default=0.5)
parser.add_argument("-b2", "--beta2", type=float, default=0.999)
parser.add_argument("-batch", "--batch_size", type=int, default=4)
parser.add_argument("-sample", "--sample_size", type=int, default=2)
parser.add_argument("-ep", "--epochs", type=int, default=400000)
parser.add_argument("-l1", "--lambda1", type=int, default=10)
parser.add_argument("-l2", "--lambda2", type=int, default=10)
parser.add_argument("-l4", "--lambda4", type=int, default=10)
parser.add_argument("-l5", "--lambda5", type=int, default=10)
parser.add_argument("-gp", "--lambda_gp", type=int, default=150)
parser.add_argument("-img", "--img_size", type=int, default=256)
parser.add_argument("-v", "--vec_size", type=int, default=17)

#args = parser.parse_args()
args = parser.parse_args(args=[])

import os
import sys

os.environ['CUDA_VISIBLE_DEVICES'] = "0"

import numpy as np

import tensorflow as tf
from keras import backend as K
#from keras.backend.tensorflow_backend import set_session
from tensorflow.python.keras.backend import set_session


# K.set_floatx('float64')
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.compat.v1.Session(config=config))

rel_gan = Relgan(args)
rel_gan.train()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 17)]         0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 256, 256, 17) 0           input_7[0][0]                    
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 256, 256, 20) 0           input_5[0][0]                    
                                                                 lambda_3[0][0]             

AttributeError: ignored

DemoTranslation

In [ ]:
import os
import sys
import random
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from PIL import Image
from keras import backend as K
from skimage import io, transform
from keras.models import load_model
from keras.backend.tensorflow_backend import set_session
from contrib.ops import SwitchNormalization
from module import *
import argparse

parser = argparse.ArgumentParser()
parser.add_argument("-d", "--device", type=str, default='0')
args = parser.parse_args()

os.environ["CUDA_VISIBLE_DEVICES"] = args.device
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

def tileAttr(x):
    x = tf.expand_dims(x, axis = 1)
    x = tf.expand_dims(x, axis = 2)
    return tf.tile(x, [1, 256, 256, 1])

def tileAttr2(x):
        x = tf.expand_dims(x, axis = 1)
        x = tf.expand_dims(x, axis = 2)
        return tf.tile(x, [1, 4, 4, 1])
    
# train_path = '/share/diskB/willy/GanExample/FaceAttributeChange_StarGAN/model/generator499.h5'

def testPic(img, gender, bangs=-1, glasses=1):
    temp3 = io.imread(img)
    tempb = transform.resize(temp3, [256,256])
    tempb = tempb[:,:,:3]
    tempb = tempb*2 - 1
    
    # imgIndex = np.load("imgIndex.npy")
# imgAttr = np.load("anno_dic.npy").item()

    new_attrs = ['5_o_Clock_Shadow', 'Bald', 'Bangs', 'Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Male', 'Mustache', 'Pale_Skin', 'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Hat', 'Young']

    def att2vec(attt):
        temp_vec = np.expand_dims(attt, axis=0)
        return temp_vec

    attrs_pos, attrs_neg = [],[]
    attr = []
    
    temp = np.zeros([17])
    # temp[new_attrs.index('Brown_Hair')] = -1
    temp[new_attrs.index('Black_Hair')] = -1
    temp[new_attrs.index('Blond_Hair')] = 1
    attr.append(temp)

    temp = np.zeros([17])
    temp[new_attrs.index('Black_Hair')] = -1
    # temp[new_attrs.index('Brown_Hair')] = -1
    # temp[new_attrs.index('Black_Hair')] = 1
    temp[new_attrs.index('Brown_Hair')] = 1
    attr.append(temp)

    temp = np.zeros([17])
    if gender==1:
        temp[new_attrs.index('Male')] = -1
    else:
        temp[new_attrs.index('Male')] = 1
    attr.append(temp)

    temp = np.zeros([17])
    #if gender==0:
    temp[new_attrs.index('Male')] = 1 
    temp[new_attrs.index('Goatee')] = 1
    temp[new_attrs.index('Mustache')] = 1
    temp[new_attrs.index('5_o_Clock_Shadow')] = 1
    attr.append(temp)

    temp = np.zeros([17])
    temp[new_attrs.index('Pale_Skin')] = 1
    attr.append(temp)

    temp = np.zeros([17])
    temp[new_attrs.index('Smiling')] = 1
    attr.append(temp)

    temp = np.zeros([17])
    temp[new_attrs.index('Bangs')] = bangs
    attr.append(temp)

    temp = np.zeros([17])
    temp[new_attrs.index('Eyeglasses')] = glasses
    attr.append(temp)

    temp = np.zeros([17])
    temp[new_attrs.index('Gray_Hair')] = 1
    temp[new_attrs.index('Young')] = -1
    attr.append(temp)
    
    for at in attr:
        att_pos = att2vec(at)
        attrs_pos.append(att_pos)

    attrs_pos = np.concatenate(attrs_pos, axis=0)    
    
    output = np.expand_dims(tempb, axis=0)
    output2 = np.tile(output, [len(attr), 1, 1, 1])
    outputs_, _ = relGan.predict([output2, attrs_pos])
    images = np.concatenate([output, outputs_], axis = 0)
    width = 1
    height = len(attr) + 1
    new_im = Image.new('RGB', (256*height, 256*width))
    for ii in range(height):
        for jj in range(width):
            index=ii*width+jj
            image = (images[index]/2+0.5)*255
            image = image.astype(np.uint8)
            new_im.paste(Image.fromarray(image,"RGB"), (256*ii,256*jj))
    ans = np.array(new_im)
    return ans

# temp3 = io.imread('/share/data/celeba-hq/celeba-256/12345.jpg')
# version = int(sys.argv[2])
# if version==-1:
#     version = len(os.listdir('img'))-2
version = 519

print(version)

train_path = './generator'+str(version)+'.h5'
#train_path = 'model/generator1511.h5'

# train_path = 'good_v0513.h5'

def orthogonal(w):
    
    w_kw = K.int_shape(w)[0]
    w_kh = K.int_shape(w)[1]
    w_w = K.int_shape(w)[2]
    w_h = K.int_shape(w)[3]
    
    temp = 0
    for i in range(w_kw):
        for j in range(w_kh):
            wwt = tf.matmul(tf.transpose(w[i,j]), w[i,j])
            mi = K.ones_like(wwt) - K.identity(wwt)
            a = wwt * mi
            a = tf.matmul(tf.transpose(a), a)
            a = a * K.identity(a)
            temp += K.sum(a)
    return 1e-4 * temp

img_shape = (256, 256, 3)
vec_shape = (17,)

imgA_input = Input(shape=img_shape)
imgB_input = Input(shape=img_shape)
vec_input_pos = Input(shape=vec_shape)
vec_input_neg = Input(shape=vec_shape)

g_out = generator(imgA_input, vec_input_pos, 256)
relGan = Model(inputs=[imgA_input, vec_input_pos], outputs=g_out)
relGan.load_weights(train_path)
relGan.summary()

lengh = 10
temp = [None]*lengh
temp[0] = testPic('test_img/j.png',0)
temp[1] = testPic('test_img/c.2.jpg',0)
temp[2] = testPic('test_img/es.png',1)
temp[3] = testPic('test_img/e.2.png',1)
temp[4] = testPic('test_img/g.2.png',1)
temp[5] = testPic('test_img/y3.png',1)
temp[6] = testPic('test_img/f1.png',1,glasses=-1)
temp[7] = testPic('test_img/j1.png',0,glasses=-1)
temp[8] = testPic('test_img/c3.png',0)
temp[9] = testPic('test_img/g3.png',1,glasses=-1)

new_im = Image.new('RGB', (256*10, 256*lengh))
for jj in tqdm(range(lengh)):
    index = jj
    image = temp[index]
    new_im.paste(Image.fromarray(image,"RGB"), (0,256*jj))
new_im.save('test_v%04d.jpg'%version)


Demo Interpolation

In [ ]:
import os
import sys
import random
import imageio
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from PIL import Image
from keras import backend as K
from skimage import io, transform
from keras.models import load_model
from keras.backend.tensorflow_backend import set_session
from contrib.ops import SwitchNormalization
from module import *

import argparse

parser = argparse.ArgumentParser()
parser.add_argument("-d", "--device", type=str, default='0')
parser.add_argument("-f", "--file", type=str, default='test_img/y3.png')
parser.add_argument("-o", "--output", type=str, default='output.gif')

args = parser.parse_args()

os.environ["CUDA_VISIBLE_DEVICES"] = args.device
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

def tileAttr(x):
    x = tf.expand_dims(x, axis = 1)
    x = tf.expand_dims(x, axis = 2)
    return tf.tile(x, [1, 256, 256, 1])

def tileAttr2(x):
        x = tf.expand_dims(x, axis = 1)
        x = tf.expand_dims(x, axis = 2)
        return tf.tile(x, [1, 4, 4, 1])

# train_path = '/share/diskB/willy/GanExample/FaceAttributeChange_StarGAN/model/generator499.h5'

def testPic(img, gender, bangs=-1, glasses=1):
    temp3 = io.imread(img)
    tempb = transform.resize(temp3, [256,256])
    tempb = tempb[:,:,:3]
    tempb = tempb*2 - 1
    
    # imgIndex = np.load("imgIndex.npy")
# imgAttr = np.load("anno_dic.npy").item()

    new_attrs = ['5_o_Clock_Shadow', 'Bald', 'Bangs', 'Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Male', 'Mustache', 'Pale_Skin', 'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Hat', 'Young']

    def att2vec(attt):
        temp_vec = np.expand_dims(attt, axis=0)
        return temp_vec

    attrs_pos, attrs_neg = [],[]
    attr = []
 
    for i in range(0,21):
        temp = np.zeros([17])
        
        temp[new_attrs.index('Smiling')] = 0.05 * i
        #temp[new_attrs.index('Gray_Hair')] = 0.07* i
        #temp[new_attrs.index('Young')] = -0.07* i
        attr.append(temp)
        
    for i in range(21,0,-1):
        temp = np.zeros([17])
        
        temp[new_attrs.index('Smiling')] = 0.05 * i
        #temp[new_attrs.index('Gray_Hair')] = 0.07* i
        #temp[new_attrs.index('Young')] = -0.07* i
        attr.append(temp)
        
    for at in attr:
        att_pos = att2vec(at)
        attrs_pos.append(att_pos)

    attrs_pos = np.concatenate(attrs_pos, axis=0)    
    
    output = np.expand_dims(tempb, axis=0)
    output2 = np.tile(output, [len(attr), 1, 1, 1])
    outputs_, _ = relGan.predict([output2, attrs_pos])
    outputs_ = np.ndarray.astype((outputs_/2+0.5)*255, np.uint8)
    imageio.mimsave('simple_'+args.output, outputs_[10:31])
    imageio.mimsave(args.output, outputs_)

# temp3 = io.imread('/share/data/celeba-hq/celeba-256/12345.jpg')
version = 519
#version = 461

train_path = './generator'+str(version)+'.h5'

print('version: ', version)
#train_path = 'model/generator1511.h5'

# train_path = 'good_v0513.h5'

img_shape = (256, 256, 3)
vec_shape = (17,)

imgA_input = Input(shape=img_shape)
imgB_input = Input(shape=img_shape)
vec_input_pos = Input(shape=vec_shape)
vec_input_neg = Input(shape=vec_shape)

g_out = generator(imgA_input, vec_input_pos, 256)
relGan = Model(inputs=[imgA_input, vec_input_pos], outputs=g_out)
relGan.load_weights(train_path)
relGan.summary()

lengh = 1
temp = [None]*lengh
temp[0] = testPic(args.file, 1, glasses=-1)
